In [69]:
import re
import spacy
import gensim
import logging
import warnings
import pandas as pd
from bs4 import BeautifulSoup
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# Enable logging for gensim - optional

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner']) 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO 

warnings.filterwarnings("ignore",category=DeprecationWarning)

from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

In [2]:
df = pd.read_json('signalmedia-1m.jsonl', lines=True)

In [3]:
df.head()

,content,id,media-type,published,source,title
0,VETERANS saluted Worcester's first ever breakf...,f7ca322d-c3e8-40d2-841f-9d7250ac72ca,News,2015-09-07T10:16:14Z,Redditch Advertiser,Worcester breakfast club for veterans gives hu...
1,New Product Gives Marketers Access to Real Key...,609772bc-0672-4db5-8516-4c025cfd54ca,News,2015-09-17T15:00:00Z,Virtualization Conference & Expo,Jumpshot Gives Marketers Renewed Visibility In...
2,Home »\rStyle » The Return Of The Nike Air Max...,1aa9d1b0-e6ba-4a48-ad0c-66552d896aac,Blog,2015-09-22T22:54:37Z,Streets Connect,The Return Of The Nike Air Max Sensation Has 8...
3,NYMag.com Daily Intelligencer Vulture The Cut ...,719699f9-47be-4bc7-969b-b53a881c95ae,Blog,2015-09-16T23:12:11Z,The Cut,This New Dating App Will Ruin Your Internet Game
4,"KUALA LUMPUR, Sept 15 (MySinchew) -- The Kuala...",a080f99a-07d9-47d1-8244-26a540017b7a,News,2015-09-15T10:17:53Z,My Sinchew,Pay up or face legal action: DBKL


In [4]:
data = df.content.values.tolist()

In [26]:
def sanitize_text(document):
    # Function to convert a document to a sequence of words,

    # 1. Remove HTML
    soup = BeautifulSoup(document, 'html5lib') # create a new bs4 object from the html data loaded
    for script in soup(["script", "style", "form"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    review_text = soup.get_text()

    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    review_text = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', review_text)

    # 3. Convert words to lower case and split them
    review_text = review_text.lower()

    # 4. Replace multiple spaces with single space
    document = re.sub(' +', ' ', review_text).strip()
    
    return [word for word in document.split() if word not in STOPWORDS]

# This method won't work for this dataset as we have html and javascript tags in 
# between text, so we have to write a custom code to perform cleanup and the text
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc), deacc=True) if word not in stop_words] for doc in texts]

def lemmatization(word_list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    doc = nlp(" ".join(word_list)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]

### The  below cell took about 4-5 hours to complete (on 1 million news article)

In [30]:
cleaned_data = [lemmatization(sanitize_text(text)) for text in data]

/data/1/anurag/poc/venv/lib/python3.4/site-packages/bs4/__init__.py:335: UserWarning: "http://open-training-courses.xyz.co.uk/course-information/?TQUK-Working-in-Confined-Spaces-(Low-+-Medium-Risk)-&-Top-Person-WATER-or-CONSTRUCTION-(QCF)-167053" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/data/1/anurag/poc/venv/lib/python3.4/site-packages/bs4/__init__.py:335: UserWarning: "http://news.yahoo.com/shoutyourabortion-campaign-explodes-social-media-222022186.html

https://www.youtube.com/watch?v=N8rxBKV87lk" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/data/1/anurag/poc/venv/lib/python3.4/site-package

/data/1/anurag/poc/venv/lib/python3.4/site-packages/bs4/__init__.py:335: UserWarning: "http://blogs.msdn.com/b/the_secure_infrastructure_guy/archive/2015/09/02/windows-server-2012-r2-wsus-issue-clients-cause-the-wsus-app-pool-to-become-unresponsive-with-http-503.aspx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/data/1/anurag/poc/venv/lib/python3.4/site-packages/bs4/__init__.py:335: UserWarning: "http://www.huffingtonpost.com/brie-gowen/30-truths-your-nurse-wants-you-to-know_b_8073324.html?ir=Healthy%20Living?ncid=newsltushpmg00000003" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/data/1/anurag/poc

In [33]:
import pickle

In [36]:
with open('news-dataset-cleaned.pkl', 'wb') as f:
    pickle.dump(cleaned_data, f)

In [42]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(cleaned_data)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in cleaned_data]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 5), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 10), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 5), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 4), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 3), (89, 1), (90, 1), (91, 1), (92, 3), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 2), (99, 1), (100, 3), (101, 6), (102, 2), (103, 1), (104, 4), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1

In [44]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('age', 1),
  ('allow', 1),
  ('andy', 1),
  ('area', 1),
  ('armed', 1),
  ('article', 3),
  ('attend', 2),
  ('attract', 1),
  ('backwards', 1),
  ('banter', 1),
  ('beer', 1),
  ('belly', 1),
  ('bent', 1),
  ('breakfast', 5),
  ('british', 1),
  ('bromsgrove', 1),
  ('carney', 3),
  ('central', 1),
  ('chap', 1),
  ('choice', 1),
  ('choose', 1),
  ('close', 1),
  ('closing', 1),
  ('club', 10),
  ('comradery', 1),
  ('country', 2),
  ('dave', 1),
  ('decide', 1),
  ('derek', 1),
  ('design', 1),
  ('designate', 1),
  ('drink', 2),
  ('droitwich', 1),
  ('eat', 1),
  ('engineer', 1),
  ('estimate', 1),
  ('ex', 1),
  ('extremely', 1),
  ('father', 1),
  ('feel', 1),
  ('food', 1),
  ('force', 3),
  ('foregate', 1),
  ('future', 1),
  ('generate', 1),
  ('germany', 1),
  ('give', 2),
  ('gloucester', 1),
  ('good', 3),
  ('grub', 1),
  ('hardman', 1),
  ('heart', 1),
  ('hill', 1),
  ('hm', 1),
  ('home', 1),
  ('hospitable', 1),
  ('hotel', 1),
  ('hour', 1),
  ('hull', 1),
  ('h

In [77]:
# model hyper-parameters
# total 110 GB of system memory
# free memory during running LDA model - 15 GB
# free memory after running LDA model - 16 GB
# The process running this jupyter notebook took 24.56% of memory = 28 GB
#
passes = 2
num_topics = 25
num_docs_to_train = 1000000

In [ ]:
%time lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:num_docs_to_train], id2word=id2word, num_topics=num_topics, passes=passes)

2019-05-08 09:57:11,667 : INFO : using symmetric alpha at 0.04
2019-05-08 09:57:11,669 : INFO : using symmetric eta at 0.04
2019-05-08 09:57:11,917 : INFO : using serial LDA version on this node
2019-05-08 09:57:16,337 : INFO : running online (multi-pass) LDA training, 25 topics, 2 passes over the supplied corpus of 1000000 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-05-08 09:57:16,340 : INFO : PROGRESS: pass 0, at document #2000/1000000
2019-05-08 09:57:22,243 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 09:57:24,607 : INFO : topic #2 (0.040): 0.014*"say" + 0.007*"year" + 0.004*"s" + 0.004*"new" + 0.003*"share" + 0.003*"minister" + 0.003*"market" + 0.003*"party" + 0.003*"government" + 0.003*"state"
2019-05-08 09:57:24,615 : INFO : topic #5 (0.040): 0.007*"say" + 0.006*"new" + 0.005*"year" + 0.005*"com" + 0.004*"s" + 0.004*"time"

2019-05-08 09:58:13,228 : INFO : topic #1 (0.040): 0.014*"phone" + 0.014*"apple" + 0.010*"video" + 0.008*"new" + 0.008*"free" + 0.007*"store" + 0.007*"watch" + 0.006*"wine" + 0.005*"live" + 0.005*"feature"
2019-05-08 09:58:13,235 : INFO : topic #3 (0.040): 0.013*"t" + 0.012*"s" + 0.007*"time" + 0.007*"think" + 0.007*"say" + 0.007*"know" + 0.007*"new" + 0.007*"good" + 0.007*"get" + 0.006*"play"
2019-05-08 09:58:13,242 : INFO : topic #21 (0.040): 0.009*"energy" + 0.008*"solar" + 0.007*"waste" + 0.006*"vehicle" + 0.006*"design" + 0.006*"say" + 0.006*"car" + 0.006*"year" + 0.006*"power" + 0.005*"new"
2019-05-08 09:58:13,248 : INFO : topic #13 (0.040): 0.013*"year" + 0.012*"rate" + 0.012*"say" + 0.011*"percent" + 0.006*"cent" + 0.006*"bank" + 0.006*"month" + 0.006*"fall" + 0.006*"price" + 0.006*"court"
2019-05-08 09:58:13,255 : INFO : topic #10 (0.040): 0.016*"game" + 0.013*"team" + 0.012*"win" + 0.011*"season" + 0.010*"say" + 0.009*"year" + 0.009*"play" + 0.007*"player" + 0.007*"yard" + 0.

2019-05-08 09:59:07,811 : INFO : topic #23 (0.040): 0.015*"film" + 0.010*"new" + 0.009*"post" + 0.009*"share" + 0.008*"movie" + 0.008*"series" + 0.008*"com" + 0.008*"award" + 0.007*"star" + 0.006*"world"
2019-05-08 09:59:07,818 : INFO : topic #0 (0.040): 0.006*"art" + 0.005*"day" + 0.005*"s" + 0.004*"home" + 0.004*"add" + 0.004*"time" + 0.004*"year" + 0.003*"artist" + 0.003*"place" + 0.003*"white"
2019-05-08 09:59:07,825 : INFO : topic #16 (0.040): 0.023*"school" + 0.017*"student" + 0.011*"say" + 0.011*"university" + 0.010*"year" + 0.010*"child" + 0.009*"work" + 0.008*"program" + 0.008*"education" + 0.007*"college"
2019-05-08 09:59:07,871 : INFO : topic diff=0.046612, rho=0.277350
2019-05-08 09:59:07,910 : INFO : PROGRESS: pass 0, at document #28000/1000000
2019-05-08 09:59:13,211 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 09:59:15,555 : INFO : topic #2 (0.040): 0.021*"say" + 0.014*"government" + 0.011*"minister" + 0.010*"party" + 0.009*"p

2019-05-08 09:59:53,676 : INFO : topic #24 (0.040): 0.037*"september" + 0.028*"police" + 0.014*"news" + 0.012*"say" + 0.011*"monday" + 0.011*"man" + 0.010*"officer" + 0.009*"net" + 0.009*"kill" + 0.008*"tuesday"
2019-05-08 09:59:53,742 : INFO : topic diff=0.034114, rho=0.229416
2019-05-08 10:00:01,747 : INFO : -10.264 per-word bound, 1229.6 perplexity estimate based on a held-out corpus of 2000 documents with 418517 words
2019-05-08 10:00:01,748 : INFO : PROGRESS: pass 0, at document #40000/1000000
2019-05-08 10:00:06,832 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 10:00:09,176 : INFO : topic #2 (0.040): 0.022*"say" + 0.014*"government" + 0.011*"minister" + 0.010*"party" + 0.009*"president" + 0.009*"state" + 0.008*"leader" + 0.006*"country" + 0.006*"vote" + 0.006*"new"
2019-05-08 10:00:09,183 : INFO : topic #22 (0.040): 0.061*"p" + 0.052*"m" + 0.051*"g" + 0.043*"d" + 0.040*"c" + 0.035*"e" + 0.031*"r" + 0.023*"b" + 0.018*"l" + 0.017*"http"
2

2019-05-08 10:00:54,336 : INFO : topic #18 (0.040): 0.030*"car" + 0.022*"trump" + 0.016*"new" + 0.009*"model" + 0.009*"donald" + 0.008*"september" + 0.008*"cruise" + 0.007*"engine" + 0.006*"seat" + 0.006*"los"
2019-05-08 10:00:54,343 : INFO : topic #2 (0.040): 0.022*"say" + 0.015*"government" + 0.011*"minister" + 0.010*"president" + 0.009*"state" + 0.009*"party" + 0.008*"leader" + 0.006*"country" + 0.006*"vote" + 0.006*"new"
2019-05-08 10:00:54,349 : INFO : topic #6 (0.040): 0.025*"say" + 0.013*"refugee" + 0.013*"country" + 0.009*"europe" + 0.008*"migrant" + 0.008*"border" + 0.008*"military" + 0.007*"european" + 0.007*"syria" + 0.007*"people"
2019-05-08 10:00:54,356 : INFO : topic #12 (0.040): 0.013*"t" + 0.012*"people" + 0.011*"time" + 0.009*"know" + 0.009*"work" + 0.008*"way" + 0.008*"think" + 0.008*"thing" + 0.008*"want" + 0.008*"good"
2019-05-08 10:00:54,363 : INFO : topic #17 (0.040): 0.026*"say" + 0.011*"court" + 0.007*"year" + 0.007*"woman" + 0.006*"case" + 0.006*"law" + 0.006*"

...

2019-05-08 11:58:03,200 : INFO : topic #15 (0.040): 0.014*"state" + 0.013*"county" + 0.011*"m" + 0.010*"day" + 0.010*"sept" + 0.010*"event" + 0.009*"road" + 0.008*"park" + 0.008*"p" + 0.007*"saturday"
2019-05-08 11:58:03,206 : INFO : topic #10 (0.040): 0.020*"game" + 0.014*"team" + 0.013*"win" + 0.013*"play" + 0.011*"season" + 0.009*"player" + 0.009*"say" + 0.008*"year" + 0.006*"run" + 0.006*"s"
2019-05-08 11:58:03,212 : INFO : topic #21 (0.040): 0.013*"energy" + 0.013*"power" + 0.010*"project" + 0.008*"high" + 0.008*"technology" + 0.008*"design" + 0.007*"use" + 0.007*"system" + 0.006*"control" + 0.006*"cost"
2019-05-08 11:58:03,255 : INFO : topic diff=0.002916, rho=0.044632
2019-05-08 11:58:03,286 : INFO : PROGRESS: pass 1, at document #812000/1000000
2019-05-08 11:58:08,174 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 11:58:10,498 : INFO : topic #24 (0.040): 0.034*"police" + 0.033*"say" + 0.018*"september" + 0.015*"man" + 0.011*"officer" +

2019-05-08 11:58:53,765 : INFO : topic #18 (0.040): 0.041*"car" + 0.024*"race" + 0.020*"trump" + 0.020*"vehicle" + 0.017*"driver" + 0.010*"drive" + 0.010*"new" + 0.009*"engine" + 0.009*"donald" + 0.008*"say"
2019-05-08 11:58:53,808 : INFO : topic diff=0.003088, rho=0.044632
2019-05-08 11:58:53,840 : INFO : PROGRESS: pass 1, at document #824000/1000000
2019-05-08 11:58:58,468 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 11:59:00,758 : INFO : topic #23 (0.040): 0.015*"new" + 0.014*"film" + 0.011*"star" + 0.011*"music" + 0.009*"movie" + 0.008*"series" + 0.007*"s" + 0.007*"tv" + 0.007*"video" + 0.007*"book"
2019-05-08 11:59:00,764 : INFO : topic #6 (0.040): 0.019*"say" + 0.015*"country" + 0.013*"refugee" + 0.008*"people" + 0.008*"war" + 0.008*"europe" + 0.008*"migrant" + 0.007*"border" + 0.007*"state" + 0.007*"military"
2019-05-08 11:59:00,771 : INFO : topic #2 (0.040): 0.026*"say" + 0.016*"government" + 0.012*"minister" + 0.011*"party" + 0.011*

2019-05-08 11:59:43,285 : INFO : topic #11 (0.040): 0.072*"market" + 0.036*"report" + 0.022*"china" + 0.021*"research" + 0.020*"industry" + 0.020*"global" + 0.013*"analysis" + 0.013*"com" + 0.012*"key" + 0.010*"product"
2019-05-08 11:59:43,291 : INFO : topic #14 (0.040): 0.032*"food" + 0.010*"eat" + 0.010*"wine" + 0.008*"restaurant" + 0.008*"beer" + 0.008*"drink" + 0.006*"coffee" + 0.006*"recipe" + 0.006*"add" + 0.006*"cup"
2019-05-08 11:59:43,297 : INFO : topic #4 (0.040): 0.034*"canada" + 0.025*"animal" + 0.025*"canadian" + 0.023*"dog" + 0.023*"farm" + 0.017*"farmer" + 0.014*"pet" + 0.014*"plant" + 0.012*"caption" + 0.011*"trade"
2019-05-08 11:59:43,303 : INFO : topic #9 (0.040): 0.017*"company" + 0.016*"com" + 0.016*"business" + 0.014*"service" + 0.010*"customer" + 0.010*"www" + 0.010*"technology" + 0.009*"solution" + 0.008*"new" + 0.007*"product"
2019-05-08 11:59:43,310 : INFO : topic #6 (0.040): 0.018*"say" + 0.015*"country" + 0.013*"refugee" + 0.009*"war" + 0.008*"people" + 0.008

2019-05-08 12:00:34,016 : INFO : topic #13 (0.040): 0.020*"year" + 0.015*"percent" + 0.015*"say" + 0.013*"rate" + 0.011*"bank" + 0.011*"market" + 0.010*"price" + 0.009*"low" + 0.008*"cent" + 0.008*"increase"
2019-05-08 12:00:34,023 : INFO : topic #11 (0.040): 0.072*"market" + 0.036*"report" + 0.022*"china" + 0.021*"research" + 0.020*"global" + 0.020*"industry" + 0.013*"com" + 0.013*"analysis" + 0.013*"key" + 0.010*"product"
2019-05-08 12:00:34,030 : INFO : topic #6 (0.040): 0.019*"say" + 0.015*"country" + 0.013*"refugee" + 0.008*"people" + 0.008*"war" + 0.008*"europe" + 0.008*"migrant" + 0.007*"border" + 0.007*"state" + 0.007*"syria"
2019-05-08 12:00:34,073 : INFO : topic diff=0.002647, rho=0.044632
2019-05-08 12:00:34,105 : INFO : PROGRESS: pass 1, at document #850000/1000000
2019-05-08 12:00:38,806 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 12:00:41,078 : INFO : topic #3 (0.040): 0.019*"say" + 0.016*"year" + 0.011*"family" + 0.011*"love"

2019-05-08 12:01:24,258 : INFO : topic #8 (0.040): 0.027*"city" + 0.017*"home" + 0.015*"new" + 0.010*"say" + 0.009*"property" + 0.009*"hotel" + 0.008*"area" + 0.007*"year" + 0.007*"house" + 0.007*"travel"
2019-05-08 12:01:24,301 : INFO : topic diff=0.003479, rho=0.044632
2019-05-08 12:01:24,333 : INFO : PROGRESS: pass 1, at document #862000/1000000
2019-05-08 12:01:29,022 : INFO : merging changes from 2000 documents into a model of 1000000 documents
2019-05-08 12:01:31,303 : INFO : topic #24 (0.040): 0.034*"police" + 0.033*"say" + 0.017*"september" + 0.015*"man" + 0.011*"officer" + 0.011*"report" + 0.009*"year" + 0.009*"kill" + 0.008*"arrest" + 0.007*"old"
2019-05-08 12:01:31,309 : INFO : topic #10 (0.040): 0.020*"game" + 0.014*"team" + 0.013*"win" + 0.013*"play" + 0.012*"season" + 0.009*"player" + 0.009*"say" + 0.008*"year" + 0.007*"run" + 0.006*"second"
2019-05-08 12:01:31,315 : INFO : topic #23 (0.040): 0.015*"new" + 0.014*"film" + 0.011*"star" + 0.011*"music" + 0.009*"movie" + 0.00

In [ ]:
# save model for later use

lda_model.save('/data/1/anurag/poc/saved_model/lda_model_25topics_2passes.model')

id2word.save('/data/1/anurag/poc/saved_model/corpora.dictionary')

In [95]:
lda_model.show_topics(num_topics=25)

[(0,
  '0.010*"black" + 0.010*"look" + 0.008*"photo" + 0.008*"image" + 0.007*"design" + 0.007*"art" + 0.007*"white" + 0.006*"color" + 0.006*"style" + 0.006*"fashion"'),
 (1,
  '0.021*"new" + 0.020*"phone" + 0.019*"app" + 0.015*"apple" + 0.012*"mobile" + 0.012*"device" + 0.011*"card" + 0.010*"store" + 0.009*"price" + 0.008*"video"'),
 (2,
  '0.026*"say" + 0.016*"government" + 0.012*"minister" + 0.011*"president" + 0.011*"party" + 0.009*"state" + 0.008*"india" + 0.008*"leader" + 0.007*"vote" + 0.007*"news"'),
 (3,
  '0.019*"say" + 0.016*"year" + 0.011*"love" + 0.011*"family" + 0.010*"old" + 0.010*"s" + 0.008*"day" + 0.008*"woman" + 0.008*"go" + 0.008*"t"'),
 (4,
  '0.034*"canada" + 0.026*"animal" + 0.025*"dog" + 0.023*"canadian" + 0.022*"farm" + 0.017*"farmer" + 0.014*"pet" + 0.013*"plant" + 0.012*"caption" + 0.011*"trade"'),
 (5,
  '0.016*"news" + 0.013*"email" + 0.013*"post" + 0.013*"information" + 0.012*"online" + 0.012*"share" + 0.012*"content" + 0.011*"service" + 0.011*"article" + 0

In [103]:
for i in range(10):
    topics = sorted(lda_model[corpus[i]], key=lambda x: (x[1]), reverse=True)
    print("Got - {} topics for document - {} - {}".format(len(topics), i, topics))
    print(sum(x[0] for x in topics))

Got - 10 topics for document - 0 - [(8, 0.14499898), (15, 0.13072945), (3, 0.121957555), (10, 0.12069983), (14, 0.10280464), (6, 0.09959555), (12, 0.09825819), (13, 0.08871568), (21, 0.05593455), (5, 0.027445216)]
107
Got - 6 topics for document - 1 - [(9, 0.5034954), (5, 0.35005507), (7, 0.05560223), (0, 0.031593893), (12, 0.03147163), (19, 0.015903011)]
52
Got - 7 topics for document - 2 - [(0, 0.28909513), (23, 0.20443146), (10, 0.19894862), (3, 0.172119), (22, 0.08118423), (5, 0.02320258), (20, 0.017921053)]
83
Got - 12 topics for document - 3 - [(3, 0.36217844), (0, 0.22422317), (23, 0.12863474), (12, 0.0684184), (5, 0.06631708), (22, 0.056103736), (1, 0.018092768), (10, 0.015853874), (14, 0.013224866), (17, 0.013060928), (16, 0.012684137), (8, 0.011509336)]
131
Got - 4 topics for document - 4 - [(2, 0.50214046), (17, 0.24435101), (21, 0.130108), (8, 0.103369586)]
48
Got - 9 topics for document - 5 - [(13, 0.5142029), (6, 0.17782713), (2, 0.07326132), (8, 0.059964687), (18, 0.0483